In [2]:
import pandas as pd

df = pd.read_excel("~/Desktop/Research/LMU_Wellness/data/Wellness_Database_May19.xlsx", sheet_name="Wellness Responses")

df = df.dropna(subset=["How well did you hydrate?"])

sore_areas = ["Neck", "Back", "Shoulders", "Chest", "Arms", "Hip Flexors", "Glutes", "Hamstrings", "Quadricps", "Adductors", "Calves", "Feet"]

for area in sore_areas:
    df[area] = df.apply(lambda row: row["How sore are you?"] / (row["Select where you are sore:"].count(",")+1) if (isinstance(row["Select where you are sore:"], str) and area in row["Select where you are sore:"]) else 0, axis=1)

# Drop the original "Select where you are sore:" column
df = df.drop("Select where you are sore:", axis=1)

columns_to_drop = ['Timestamp', 'Athlete ID #', 'Data ID', 'Week ID', 'Week ID Refined', 'Date Value', 'Year ID', 'Season ID', 'Injury Refined', 'Position', 'Classification', 'Stress RA', 'Stress StdDev', 'Stress Z-Score', 'Stress Wellness Score', 'Sleep Quality RA', 'Sleep Quality StdDev', 'Sleep Quality Z-Score', 'Sleep Quality Wellness Score', 'Sleep Quantity RA', 'Sleep Quantity StdDev', 'Sleep Quantity Z-Score', 'Sleep Quantity Wellness Score', 'Soreness RA', 'Soreness StdDev', 'Soreness Z-Score', 'Soreness Wellness Score', 'Hydrate RA', 'Hydrate StdDev', 'Hydrate Z-Score', 'Hydrate Wellness Score', 'Fuel RA', 'Fuel StdDev', 'Fuel Z-Score', 'Fuel Wellness Score', 'Readiness Score']
df = df.drop(columns_to_drop, axis=1)

df['No Injury'] = ((df['What is your injury status?'] == 'Full = I have no injury').astype(int))*10
df['Some Injury'] = ((df['What is your injury status?'] == 'Limited = I need a modification during lift / practice').astype(int))*10
df['Injury'] = ((df['What is your injury status?'] == 'Out = I have an injury').astype(int))*10

# Drop the original column
df = df.drop(['What is your injury status?', "Athlete Name"], axis=1)
df = df.rename(columns={"Athlete Name": "AthleteName"})

In [11]:
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence

# Prepare the data
X = df.drop("What is your readiness score? ", axis=1)  # Features
y = df["What is your readiness score? "]  # Target variable

# Add a constant term to the features
X = sm.add_constant(X)

# Fit the ordinary least squares (OLS) model
model = sm.OLS(y, X)
results = model.fit()

# Compute leverage values
leverage = OLSInfluence(results).hat_matrix_diag

# Compute Cook's distance
cooks_d = OLSInfluence(results).cooks_distance[0]

# Set a threshold for identifying influential points
threshold = 4 / len(X)  # You can adjust the threshold as needed

# Identify influential points based on Cook's distance and leverage
influential_points = np.where((cooks_d > threshold) | (leverage > np.mean(leverage) + 2 * np.std(leverage)))

# Remove values
for i in range(len(influential_points[0])):
    df = df.drop(index = (influential_points[0][i]+1061) ) 

/Users/gbutts/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [12]:
X = df.drop("What is your readiness score? ", axis=1)  # Features
y = df["What is your readiness score? "]  # Target variable
feature_names = X.columns

In [13]:
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression

# Assuming you have your feature matrix X and target variable y loaded

# Define the number of features (K) you want to select
num_features_to_select = 'all'

# Initialize the feature selector
selector = SelectKBest(score_func=f_regression, k=num_features_to_select)

# Fit the selector to your data
selector.fit(X, y)

# Get the scores of each feature
feature_scores = selector.scores_

# Get the indices of the selected features
selected_feature_indices = selector.get_support(indices=True)

# Get the names of the selected features (assuming you have feature names available)
selected_feature_names = [feature_names[i] for i in selected_feature_indices]

# Print the scores and names of the selected features
for score, name in zip(feature_scores, selected_feature_names):
    print(f"Feature: {name}, Score: {score:.4f}")

Feature: How stressed are you?, Score: 61.5003
Feature: How well did you sleep?, Score: 31.8825
Feature: How many hours did you sleep?, Score: 19.1834
Feature: How sore are you?, Score: 134.8101
Feature: How well did you hydrate?, Score: 0.2887
Feature: How well did you fuel?, Score: 5.8558
Feature: Neck, Score: 0.0067
Feature: Back, Score: 2.4116
Feature: Shoulders, Score: 13.5853
Feature: Chest, Score: 0.0379
Feature: Arms, Score: 1.4422
Feature: Hip Flexors, Score: 0.1670
Feature: Glutes, Score: 22.2383
Feature: Hamstrings, Score: 21.7082
Feature: Quadricps, Score: 3.7976
Feature: Adductors, Score: 0.0323
Feature: Calves, Score: 0.0332
Feature: Feet, Score: 1.4635
Feature: No Injury, Score: 9.7937
Feature: Some Injury, Score: 9.7937
Feature: Injury, Score: nan


/Users/gbutts/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


In [ ]:
#List of best variables

In [ ]:
features = ['How sore are you?', 'How stressed are you?', 'How well did you sleep?', 'Glutes', 'Hamstrings', 'How many hours did you sleep?', 'Shoulders', 'No Injury']